In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
books=pd.read_csv('/kaggle/input/book-recommendation-dataset/Books.csv')
users=pd.read_csv('/kaggle/input/book-recommendation-dataset/Users.csv')
ratings=pd.read_csv('/kaggle/input/book-recommendation-dataset/Ratings.csv')

In [ ]:
books.head()

In [ ]:
users.head()

In [ ]:
ratings.head()

In [ ]:
print(books.shape)
print(ratings.shape)
print(users.shape)

In [ ]:
books.isnull().sum()

**##Popularity Based Recommendation System**

In [ ]:
ratings_with_name=ratings.merge(books,on='ISBN')

In [ ]:
ratings_with_name.shape

In [ ]:
ratings_with_name

In [ ]:
ratings_with_name.groupby('Book-Title')

In [ ]:
ratings_with_name.groupby('Book-Title').count()

In [ ]:
ratings_with_name.groupby('Book-Title').count()['Book-Rating']

In [ ]:
num_rating_df=ratings_with_name.groupby('Book-Title').count()['Book-Rating'].reset_index()
num_rating_df.rename(columns={'Book-Rating':'num_ratings'},inplace=True)

In [ ]:
ratings_with_name

In [ ]:
num_rating_df

In [ ]:
avg_rating_df=ratings_with_name.groupby('Book-Title').mean()['Book-Rating'].reset_index()
avg_rating_df.rename(columns={'Book-Rating':'avg_ratings'},inplace=True)
avg_rating_df

In [ ]:
popular_df=num_rating_df.merge(avg_rating_df,on='Book-Title')
popular_df

In [ ]:
popular_df[popular_df['num_ratings']>=250]

In [ ]:
popular_df[popular_df['num_ratings']>=250].sort_values('avg_ratings',ascending=False)

In [ ]:
popular_df=popular_df[popular_df['num_ratings']>=250].sort_values('avg_ratings',ascending=False).head(50)

In [ ]:
popular_df

In [ ]:
popular_df.merge(books,on='Book-Title')

In [ ]:
popular_df.merge(books,on='Book-Title').drop_duplicates('Book-Title')

In [ ]:
popular_df.merge(books,on='Book-Title').drop_duplicates('Book-Title').shape

In [ ]:
popular_df=popular_df.merge(books,on='Book-Title').drop_duplicates('Book-Title')[['Book-Title','Book-Author','Image-URL-M','num_ratings','avg_ratings']]

**Collaborative Filtering Based Recommender System**

In [ ]:
x=ratings_with_name.groupby('User-ID').count()['Book-Rating']>200
x[x]

In [ ]:
padhe_likhe_users=x[x].index
padhe_likhe_users

In [ ]:
ratings_with_name['User-ID'].isin(padhe_likhe_users)

In [ ]:
filtered_rating=ratings_with_name[ratings_with_name['User-ID'].isin(padhe_likhe_users)]
filtered_rating

In [ ]:
y=filtered_rating.groupby('Book-Title').count()['Book-Rating']>=50
famous_books=y[y].index

In [ ]:
final_ratings=filtered_rating[filtered_rating['Book-Title'].isin(famous_books)]

In [ ]:
pt=final_ratings.pivot_table(index='Book-Title',columns='User-ID',values='Book-Rating')

In [ ]:
pt

In [ ]:
pt.fillna(0,inplace=True)

In [ ]:
pt

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity_scores=cosine_similarity(pt)
similarity_scores

In [ ]:
similarity_scores.shape

In [ ]:
def recommend (book_name):
    #fetch index
    index=np.where(pt.index==book_name)[0][0]
    similar_items=sorted(list(enumerate(similarity_scores[index])),key=lambda x:x[1],reverse=True)[1:6]
    
    data=[]

    for i in similar_items:
        item=[]
        temp_df=books[books['Book-Title']==pt.index[i[0]]]
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Title'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Author'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Image-URL-M'].values))
        
        data.append(item)
        
    return data

In [ ]:
recommend('1984')

In [ ]:
import pickle
pickle.dump(popular_df,open('popular.pkl','wb'))

In [ ]:
popular_df

In [ ]:
pickle.dump(pt,open('pt.pkl','wb'))
pickle.dump(books,open('books.pkl','wb'))
pickle.dump(similarity_scores,open('similarity_scores.pkl','wb'))